In [1]:
import numpy as np
import pandas as pd
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import Data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

X_train = train.drop('SalePrice', axis = 1)
X_test = test
y_train = train['SalePrice']

## Combine train and test data

In [3]:
# 合并一下train和test，两个一起处理，之后再分开
X = X_train.append(X_test, ignore_index = True)
# Drop id
X.drop('Id', axis = 1, inplace = True)

print(X_train.shape, X_test.shape, X.shape)
X.head()

(1460, 80) (1459, 80) (2919, 79)


MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0          60       RL         65.0     8450   Pave   NaN      Reg   
1          20       RL         80.0     9600   Pave   NaN      Reg   
2          60       RL         68.0    11250   Pave   NaN      IR1   
3          70       RL         60.0     9550   Pave   NaN      IR1   
4          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC Fence  \
0         Lvl    AllPub    Inside  ...           0        0    NaN   NaN   
1         Lvl    AllPub       FR2  ...           0        0    NaN   NaN   
2         Lvl    AllPub    Inside  ...           0        0    NaN   NaN   
3         Lvl    AllPub    Corner  ...           0        0    NaN   NaN   
4         Lvl    AllPub       FR2  ...           0        0    NaN   NaN   

  MiscFeature MiscVal  MoSold  YrSold  SaleType  SaleCondition  
0         NaN       0       2    2008        WD         Normal  
1         NaN       0       5    2007        WD         Normal  
2         NaN       0       9    2008        WD         Normal  
3         NaN       0       2    2006        WD        Abnorml  
4         NaN       0      12    2008        WD         Normal  

[5 rows x 79 columns]

In [4]:
# 0 - 1459 是train
# 1460 开始是test

## Numerical columns and Categorical columns

In [5]:
# Numerical Columns
num_col = X.columns[(X.dtypes != 'object').values.tolist()].tolist()
cat_col = X.columns[(X.dtypes == 'object').values.tolist()].tolist()

In [6]:
# Check if we catch every column
print('num_col:', len(num_col))
print('cat_col:', len(cat_col))
print('X:', len(X.columns))

num_col: 36
cat_col: 43
X: 79


# Data preprocessing

## Missingness

In [7]:
missing = X.isnull().sum(axis = 0)
missing = (missing[missing != 0])/X.shape[0]
pd.DataFrame(missing, columns = ['Missing Percent']).sort_values('Missing Percent', ascending = False)

Missing Percent
PoolQC               0.996574
MiscFeature          0.964029
Alley                0.932169
Fence                0.804385
FireplaceQu          0.486468
LotFrontage          0.166495
GarageFinish         0.054471
GarageQual           0.054471
GarageCond           0.054471
GarageYrBlt          0.054471
GarageType           0.053786
BsmtExposure         0.028092
BsmtCond             0.028092
BsmtQual             0.027749
BsmtFinType2         0.027407
BsmtFinType1         0.027064
MasVnrType           0.008222
MasVnrArea           0.007879
MSZoning             0.001370
BsmtFullBath         0.000685
BsmtHalfBath         0.000685
Functional           0.000685
Utilities            0.000685
GarageArea           0.000343
GarageCars           0.000343
Electrical           0.000343
KitchenQual          0.000343
TotalBsmtSF          0.000343
BsmtUnfSF            0.000343
BsmtFinSF2           0.000343
BsmtFinSF1           0.000343
Exterior2nd          0.000343
Exterior1st          0.000343
SaleType             0.000343

### Impute missingness Type 1 (miss是因为没有）

In [8]:
# Alley
X['Alley'].fillna('No_Alley', inplace = True)

# Bsmt相关
Bsmt_col = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
for i in Bsmt_col:
    X[i].fillna('No_Bsmt', inplace = True)

# Garage相关
Garage_col = ['GarageType','GarageFinish', 'GarageQual', 'GarageCond']
for k in Garage_col:
    X[k].fillna('No_Garage', inplace = True)
    
Garage_col2 = ['GarageYrBlt','GarageCars', 'GarageArea']
for a in Garage_col2:
    X[a].fillna(0, inplace = True)

# FireplaceQu
X['FireplaceQu'].fillna('No_Fireplace', inplace = True)

# PoolQC
X['PoolQC'].fillna('No_Pool', inplace = True)

# Fence
X['Fence'].fillna('No_Fence', inplace = True)

# MiscFeature
X['MiscFeature'].fillna('No_MiscFeature', inplace = True)

In [9]:
# Check missingness again
missing = X.isnull().sum(axis = 0)
missing = (missing[missing != 0])/X.shape[0]
pd.DataFrame(missing, columns = ['Missing Percent']).sort_values('Missing Percent', ascending = False)

Missing Percent
LotFrontage          0.166495
MasVnrType           0.008222
MasVnrArea           0.007879
MSZoning             0.001370
Utilities            0.000685
Functional           0.000685
BsmtHalfBath         0.000685
BsmtFullBath         0.000685
Electrical           0.000343
KitchenQual          0.000343
BsmtFinSF2           0.000343
TotalBsmtSF          0.000343
BsmtUnfSF            0.000343
BsmtFinSF1           0.000343
Exterior2nd          0.000343
Exterior1st          0.000343
SaleType             0.000343

### Impute missingness Type 2 (median)

In [12]:
missing_col = missing.index.tolist()
num_missing_col = []
for i in missing_col:
    if i in num_col:
        num_missing_col.append(i)
num_missing_col

['LotFrontage',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath']

In [13]:
for i in num_missing_col:
    X[i] = X.groupby(['Neighborhood'])[i].apply(lambda x: x.fillna(x.median()))

### Impute missingness Type 3 (mode)

In [14]:
missing_col = missing.index.tolist()
cat_missing_col = []
for i in missing_col:
    if i in cat_col:
        cat_missing_col.append(i)
cat_missing_col

['MSZoning',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'Electrical',
 'KitchenQual',
 'Functional',
 'SaleType']

In [15]:
for i in cat_missing_col:
    X[i] = X.groupby(['Neighborhood'])[i].apply(lambda x:x.fillna(x.value_counts().index[0]))

### Check missingness again

In [16]:
X.isnull().sum().sum()

0

In [17]:
# OK everything clear!

## Nominal / Ordinal

### Onehot

In [18]:
onehot_col = ['MSSubClass','MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','Neighborhood',
              'Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
              'Exterior2nd','MasVnrType','Foundation','Heating','CentralAir','Electrical','GarageType',
              'MiscFeature','SaleType','SaleCondition']

In [19]:
X = pd.get_dummies(X, columns = onehot_col, drop_first=True)

In [20]:
X.head()

LotFrontage  LotArea LandSlope  OverallQual  OverallCond  YearBuilt  \
0         65.0     8450       Gtl            7            5       2003   
1         80.0     9600       Gtl            6            8       1976   
2         68.0    11250       Gtl            7            5       2001   
3         60.0     9550       Gtl            7            5       1915   
4         84.0    14260       Gtl            8            5       2000   

   YearRemodAdd  MasVnrArea ExterQual ExterCond  ... SaleType_ConLI  \
0          2003       196.0        Gd        TA  ...              0   
1          1976         0.0        TA        TA  ...              0   
2          2002       162.0        Gd        TA  ...              0   
3          1970         0.0        TA        TA  ...              0   
4          2000       350.0        Gd        TA  ...              0   

  SaleType_ConLw SaleType_New SaleType_Oth  SaleType_WD SaleCondition_AdjLand  \
0              0            0            0            1                     0   
1              0            0            0            1                     0   
2              0            0            0            1                     0   
3              0            0            0            1                     0   
4              0            0            0            1                     0   

   SaleCondition_Alloca  SaleCondition_Family  SaleCondition_Normal  \
0                     0                     0                     1   
1                     0                     0                     1   
2                     0                     0                     1   
3                     0                     0                     0   
4                     0                     0                     1   

  SaleCondition_Partial  
0                     0  
1                     0  
2                     0  
3                     0  
4                     0  

[5 rows x 218 columns]

### Order

In [21]:
X['LandSlope'].replace({'Sev':2, 'Mod':1, 'Gtl':0}, inplace = True)

X['BsmtExposure'].replace({'Gd':4, 'Av':3, 'Mn':2, 'No':1, 'No_Bsmt':0}, inplace = True)

X['Functional'].replace({'Typ':7, 'Min1':6, 'Min2':5, 'Mod':4, 'Maj1':3, 
                        'Maj2':2, 'Sev':1, 'Sal':0}, inplace = True)

X['FireplaceQu'].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No_Fireplace':0}, inplace = True)

X['GarageFinish'].replace({'Fin':3, 'RFn':2, 'Unf':1, 'No_Garage':0}, inplace = True)

X['PavedDrive'].replace({'Y':2, 'P':1, 'N':0}, inplace = True)

X['PoolQC'].replace({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No_Pool':0}, inplace = True)

X['Fence'].replace({'GdPrv':4, 'MnPrv':3, 'GdWo':2, 'MnWw':1, 'No_Fence':0}, inplace = True)

replace_dict1 = {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0}
type1_col = ['ExterQual', 'ExterCond', 'HeatingQC','KitchenQual']
for i in type1_col:
    X[i].replace(replace_dict1, inplace = True)

replace_dict2 = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No_Bsmt':0}
type2_col = ['BsmtQual', 'BsmtCond']
for i in type2_col:
    X[i].replace(replace_dict2, inplace = True)

replace_dict3 = {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'No_Bsmt':0}
type3_col = ['BsmtFinType1','BsmtFinType2']
for i in type3_col:
    X[i].replace(replace_dict3, inplace = True)

replace_dict4 = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No_Garage':0}
type4_col = ['GarageQual','GarageCond']
for i in type4_col:
    X[i].replace(replace_dict4, inplace = True)

In [22]:
# Check if we convert everything to numerical values
(X.dtypes).loc[(X.dtypes == object).tolist()]


Series([], dtype: object)

In [23]:
# Everything clear!

# Feature Engineering

In [24]:
print(X.shape)
X.head()

(2919, 218)


LotFrontage  LotArea  LandSlope  OverallQual  OverallCond  YearBuilt  \
0         65.0     8450          0            7            5       2003   
1         80.0     9600          0            6            8       1976   
2         68.0    11250          0            7            5       2001   
3         60.0     9550          0            7            5       1915   
4         84.0    14260          0            8            5       2000   

   YearRemodAdd  MasVnrArea  ExterQual  ExterCond  ...  SaleType_ConLI  \
0          2003       196.0          3          2  ...               0   
1          1976         0.0          2          2  ...               0   
2          2002       162.0          3          2  ...               0   
3          1970         0.0          2          2  ...               0   
4          2000       350.0          3          2  ...               0   

   SaleType_ConLw  SaleType_New  SaleType_Oth  SaleType_WD  \
0               0             0             0            1   
1               0             0             0            1   
2               0             0             0            1   
3               0             0             0            1   
4               0             0             0            1   

   SaleCondition_AdjLand  SaleCondition_Alloca  SaleCondition_Family  \
0                      0                     0                     0   
1                      0                     0                     0   
2                      0                     0                     0   
3                      0                     0                     0   
4                      0                     0                     0   

   SaleCondition_Normal  SaleCondition_Partial  
0                     1                      0  
1                     1                      0  
2                     1                      0  
3                     0                      0  
4                     1                      0  

[5 rows x 218 columns]

### Generate new features

In [25]:
################## 所有关于时间的 ##################
X['Sell_from_Build'] = X['YrSold'] - X['YearBuilt']
X['Sell_from_Remod'] = X['YrSold'] - X['YearRemodAdd']
X['Sell_from_Garage'] = X['YrSold'] - X['GarageYrBlt']
X['Remod_from_Build'] = X['YearRemodAdd'] - X['YearBuilt']
X['Garage_from_Build'] = X['GarageYrBlt'] - X['YearBuilt']
# 注意那个GarageYrBlt没有Garage的就给我弄成0了！！

# Avoid negative number for ages
for i in ['Sell_from_Build','Sell_from_Remod','Sell_from_Garage','Remod_from_Build','Garage_from_Build']:
    X[i] = X[i].map(lambda x: 0 if x < 0 else x)

# Update num_col
num_col += ['Sell_from_Build','Sell_from_Remod','Sell_from_Garage','Remod_from_Build','Garage_from_Build']

In [27]:
################## 所有关于Bath的 ##################
X['BsmtBath'] = X['BsmtFullBath'] + 0.5 * X['BsmtHalfBath']
X['Bath'] = X['FullBath'] + 0.5 * X['HalfBath']

# Update num_col
num_col += ['BsmtBath', 'Bath']

In [29]:
################## 所有Cond+Qual ##################
X['Overall'] = X['OverallCond'] + X['OverallQual']
X['Exter'] = X['ExterCond'] + X['ExterQual']
X['Bsmt'] = X['BsmtCond'] + X['BsmtQual']
X['Garage'] = X['GarageQual'] + X['GarageCond']

# Update num_col
num_col += ['Overall', 'Exter', 'Bsmt', 'Garage']

In [30]:
print(X.shape)
X.head()

(2919, 229)


LotFrontage  LotArea  LandSlope  OverallQual  OverallCond  YearBuilt  \
0         65.0     8450          0            7            5       2003   
1         80.0     9600          0            6            8       1976   
2         68.0    11250          0            7            5       2001   
3         60.0     9550          0            7            5       1915   
4         84.0    14260          0            8            5       2000   

   YearRemodAdd  MasVnrArea  ExterQual  ExterCond  ...  Sell_from_Remod  \
0          2003       196.0          3          2  ...                5   
1          1976         0.0          2          2  ...               31   
2          2002       162.0          3          2  ...                6   
3          1970         0.0          2          2  ...               36   
4          2000       350.0          3          2  ...                8   

   Sell_from_Garage  Remod_from_Build  Garage_from_Build  BsmtBath  Bath  \
0               5.0                 0                0.0       1.0   2.5   
1              31.0                 0                0.0       0.5   2.0   
2               7.0                 1                0.0       1.0   2.5   
3               8.0                55               83.0       1.0   1.0   
4               8.0                 0                0.0       1.0   2.5   

   Overall  Exter  Bsmt  Garage  
0       12      5     7       6  
1       14      4     7       6  
2       12      5     7       6  
3       12      4     7       6  
4       13      5     7       6  

[5 rows x 229 columns]

### Skewness 

In [31]:
num_col.remove('MSSubClass')

In [32]:
col_name = []
col_skew = []
for i in num_col:
    col_name.append(i)
    col_skew.append(X[i].skew())
skew_df = pd.DataFrame({'Column':col_name, 'Skewness':col_skew}).sort_values('Skewness', ascending = False)
skew_df.head()

Column   Skewness
32       MiscVal  21.958480
31      PoolArea  16.907017
1        LotArea  12.829025
13  LowQualFinSF  12.094977
29     3SsnPorch  11.381914

In [33]:
skewed_features = skew_df[abs(skew_df['Skewness']) >= 0.5]['Column'].tolist()

In [34]:
X[skewed_features] = np.log1p(X[skewed_features])

### RobustScaler

In [37]:
from sklearn import preprocessing
sd = preprocessing.RobustScaler()

In [38]:
X[num_col] = sd.fit_transform(X[num_col])

In [39]:
print(X.shape)
X.head()

(2919, 229)


LotFrontage   LotArea  LandSlope  OverallQual  OverallCond  YearBuilt  \
0    -0.257516 -0.256995          0          0.5     0.000000   0.628146   
1     0.464671  0.035356          0          0.0     2.630317   0.063243   
2    -0.100761  0.398758          0          0.5     0.000000   0.586563   
3    -0.535329  0.023391          0          0.5     0.000000  -1.241957   
4     0.634651  0.941998          0          1.0     0.000000   0.565756   

   YearRemodAdd  MasVnrArea  ExterQual  ExterCond  ...  Sell_from_Remod  \
0      0.256410    1.034716          3          2  ...        -0.256410   
1     -0.435897    0.000000          2          2  ...         0.410256   
2      0.230769    0.997612          3          2  ...        -0.230769   
3     -0.589744    0.000000          2          2  ...         0.538462   
4      0.179487    1.147836          3          2  ...        -0.179487   

   Sell_from_Garage  Remod_from_Build  Garage_from_Build  BsmtBath      Bath  \
0         -0.877351          0.000000           0.000000  1.000000  0.333333   
1          0.016962          0.000000           0.000000  0.584963  0.000000   
2         -0.723659          0.235409           0.000000  1.000000  0.333333   
3         -0.660734          1.367103           4.430817  1.000000 -0.666667   
4         -0.660734          0.000000           0.000000  1.000000  0.333333   

    Overall  Exter  Bsmt  Garage  
0  0.000000    1.0   0.0     0.0  
1  0.928318    0.0   0.0     0.0  
2  0.000000    1.0   0.0     0.0  
3  0.000000    0.0   0.0     0.0  
4  0.480750    1.0   0.0     0.0  

[5 rows x 229 columns]

### Delete skewed columns

In [40]:
to_remove = X[num_col].skew()[(abs(X[num_col].skew()) > 1)].index.tolist()

In [41]:
to_remove

['LotFrontage',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'LowQualFinSF',
 'BsmtHalfBath',
 'KitchenAbvGr',
 'GarageYrBlt',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'Garage_from_Build',
 'Overall',
 'Bsmt',
 'Garage']

In [42]:
X.drop(to_remove, axis = 1, inplace = True)

### Delete correlated columns

In [43]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  28


In [44]:
X.drop(labels = corr_features, axis = 1, inplace = True)

## Back to Train and Test

In [45]:
X_train = X[:1460]
X_test = X[1460:]
y_train = np.log(y_train)

In [46]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(2919, 184)
(1460, 184)
(1459, 184)


### Drop outliers

In [47]:
X_train.drop([1298, 523], inplace = True)
y_train.drop([1298, 523], inplace = True)

# Feature Selection

In [48]:
import xgboost
from sklearn.feature_selection import RFECV
from sklearn.metrics import make_scorer
def neg_rmse(y_true, y_pred):
    return (-1) * np.sqrt(np.mean((y_true - y_pred) ** 2))


scorer = make_scorer(neg_rmse)
xgb_reg = xgboost.XGBRegressor()
selector = RFECV(xgb_reg, cv = 5, n_jobs = -1, scoring = scorer)
selector = selector.fit(X_train, y_train)



selected_feat = X_train.columns.values[selector.support_]
print('There are {} selected features'.format(selector.n_features_))
selected_feat

There are 63 selected features


array(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'ExterCond', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', '1stFlrSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'KitchenQual',
       'Functional', 'Fireplaces', 'GarageFinish', 'GarageCars',
       'GarageQual', 'PavedDrive', 'OpenPorchSF', 'MSSubClass_30',
       'MSSubClass_60', 'MSSubClass_90', 'MSSubClass_160', 'MSZoning_FV',
       'MSZoning_RL', 'LotConfig_CulDSac', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_Crawfor',
       'Neighborhood_Edwards', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NWAmes',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown',
       'Neighborhood_Sawyer', 'Neighborhood_StoneBr', 'Condition1_Norm',
       'Condition1_PosA', 'Condition1_RRAe', 'Condition1_RRAn',
       'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'RoofSt

## Get X and y

In [49]:
y_train = np.log(train['SalePrice'])
y_train.drop([1298, 523], inplace = True)

In [50]:
X_train_selected = X_train[selected_feat]
X_test_selected = X_test[selected_feat]

In [51]:
########################## 到这边应该都没有问题了 ###########################

# Model Selection

In [117]:
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score
import xgboost

def neg_rmse(y_true, y_pred):
    return (-1) * np.sqrt(np.mean((y_true - y_pred) ** 2))


MLA = [
    #Linear Model
    linear_model.Lasso(),
    linear_model.Ridge(
    alpha=0.98, 
    solver='sparse_cg'
),
    linear_model.ElasticNet(),

    #Kernel Ridge
    KernelRidge(),
    
    #Tree
    tree.DecisionTreeRegressor(),
    
    #Ensemble
    ensemble.RandomForestRegressor(
),
    ensemble.GradientBoostingRegressor(
    n_estimators=200,
    max_depth = 3,
    min_samples_split = 3,
    max_features = 'auto',
    criterion = 'mse'
    
),
    
    #XGB
    xgboost.XGBRegressor(
    eta = 0.1,
    max_depth = 3,
    gamma = 0,
    alpha = 0,
    tree_method = 'auto' 
)
]
scorer = make_scorer(neg_rmse)
df_columns = ['Name', 'Parameters', 'CV score mean']
df = pd.DataFrame(columns = df_columns)
from sklearn.model_selection import cross_validate
for i in np.arange(len(MLA)):
    model = MLA[i]
    model_name = model.__class__.__name__
    model_parameters = str(model.get_params())
    cv_results = (cross_val_score(model, X_train_selected, y_train, cv = 5, scoring=scorer)).mean()
    df.loc[i,:] = [model_name, model_parameters, cv_results]
    

df.sort_values('CV score mean', ascending = False)

Name  \
1                      Ridge   
6  GradientBoostingRegressor   
7               XGBRegressor   
5      RandomForestRegressor   
4      DecisionTreeRegressor   
0                      Lasso   
2                 ElasticNet   
3                KernelRidge   

                                          Parameters CV score mean  
1  {'alpha': 0.98, 'copy_X': True, 'fit_intercept...     -0.115388  
6  {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...      -0.12091  
7  {'objective': 'reg:squarederror', 'base_score'...     -0.124188  
5  {'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...     -0.136931  
4  {'ccp_alpha': 0.0, 'criterion': 'mse', 'max_de...     -0.201158  
0  {'alpha': 1.0, 'copy_X': True, 'fit_intercept'...     -0.399471  
2  {'alpha': 1.0, 'copy_X': True, 'fit_intercept'...     -0.399471  
3  {'alpha': 1, 'coef0': 1, 'degree': 3, 'gamma':...     -0.648773

- Ridge
- GradientBoosting Regressor
- XGBregressor
- RandomForestRegressor

# Model tuning

In [65]:
# Ridge
ridge_reg = linear_model.Ridge(
    alpha=0.98, 
    solver='sparse_cg'
)
parameters = {
    'alpha':[0.9, 0.95, 1, 0.96, 0.97, 0.94, 0.98],
    'solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}
cv = GridSearchCV(estimator=ridge_reg, param_grid=parameters, scoring='r2', n_jobs = -1, cv = 5)

cv.fit(X_train_selected, y_train)

cv.best_params_

In [69]:
# GradientBoostingRegressor
gb_reg = ensemble.GradientBoostingRegressor(
    n_estimators=200,
    max_depth = 3,
    min_samples_split = 3,
    max_features = 'auto',
    criterion = 'mse'
    
)
parameters = {
    'criterion':['mse', 'mae']
}
cv = GridSearchCV(estimator=gb_reg, param_grid=parameters, scoring=scorer, n_jobs = -1, cv = 5)

cv.fit(X_train_selected, y_train)

cv.best_params_

{'criterion': 'mse'}

In [71]:
# # RandomForestRegressor
# rf_reg = ensemble.RandomForestRegressor(
#      n_estimators = 400, 
#      max_depth = 45,
#      max_features = 'sqrt',
#      min_samples_split = 3
# )
# parameters = {
#     'min_samples_split':[2,3,4]
# }
# cv = GridSearchCV(estimator=rf_reg, param_grid=parameters, scoring='r2', n_jobs = -1, cv = 5)

# cv.fit(X_train_selected, y_train)

# cv.best_params_

In [79]:
# XGBregressor
xgb_reg = xgboost.XGBRegressor(
    eta = 0.1,
    max_depth = 3,
    gamma = 0,
    alpha = 0,
    tree_method = 'auto' 
)
parameters = {
    'process_type':['default', 'update']
}
cv = GridSearchCV(estimator = xgb_reg, param_grid=parameters, scoring='r2', n_jobs = -1, cv = 5)

cv.fit(X_train_selected, y_train)

cv.best_params_

{'process_type': 'default'}

# Train the Models

In [118]:
from sklearn import ensemble
from sklearn import linear_model
import xgboost

xgb_reg = xgboost.XGBRegressor(
    eta = 0.1,
    max_depth = 3,
    gamma = 0,
    alpha = 0,
    tree_method = 'auto' 
)

ridge_reg = linear_model.Ridge(
    alpha=0.98, 
    solver='sparse_cg'
)

gb_reg = ensemble.GradientBoostingRegressor(
    n_estimators=200,
    max_depth = 3,
    min_samples_split = 3,
    max_features = 'auto',
    criterion = 'mse'
)


# Train the models 
gb_reg.fit(X_train_selected, y_train)
ridge_reg.fit(X_train_selected, y_train)
xgb_reg.fit(X_train_selected, y_train)

XGBRegressor(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='auto', validate_parameters=1, verbosity=None)

# Model Ensemble

In [85]:
gb_pred = gb_reg.predict(X_train_selected)
ridge_pred = ridge_reg.predict(X_train_selected)
xgb_pred = xgb_reg.predict(X_train_selected)

In [86]:
new_X_train = pd.DataFrame({'gb':gb_pred, 'ridge':ridge_pred, 'xgb':xgb_pred})
new_X_train

gb      ridge        xgb
0     12.220987  12.258087  12.243120
1     12.020745  12.131502  12.019635
2     12.250122  12.336953  12.288778
3     11.946824  12.123184  12.074192
4     12.566651  12.588755  12.608984
...         ...        ...        ...
1453  12.098706  12.073338  12.024165
1454  12.221661  12.272109  12.221566
1455  12.503866  12.452896  12.423336
1456  11.829861  11.834426  11.806992
1457  11.889643  11.918660  11.872959

[1458 rows x 3 columns]

In [111]:
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score
import xgboost

def neg_rmse(y_true, y_pred):
    return (-1) * np.sqrt(np.mean((y_true - y_pred) ** 2))


MLA = [
    #Linear Model
    linear_model.Lasso(),
    linear_model.Ridge(),
    linear_model.ElasticNet(),

    #Kernel Ridge
    KernelRidge(alpha=0.01),
    
    #Tree
    tree.DecisionTreeRegressor(),
    
    #Ensemble
    ensemble.RandomForestRegressor(),
    ensemble.GradientBoostingRegressor(),
    
    #XGB
    xgboost.XGBRegressor()
]
scorer = make_scorer(neg_rmse)
df_columns = ['Name', 'Parameters', 'CV score mean']
df = pd.DataFrame(columns = df_columns)
from sklearn.model_selection import cross_validate
for i in np.arange(len(MLA)):
    model = MLA[i]
    model_name = model.__class__.__name__
    model_parameters = str(model.get_params())
    cv_results = (cross_val_score(model, new_X_train, y_train, cv = 5, scoring=scorer)).mean()
    df.loc[i,:] = [model_name, model_parameters, cv_results]
    

df.sort_values('CV score mean', ascending = False)

Name  \
3                KernelRidge   
1                      Ridge   
6  GradientBoostingRegressor   
5      RandomForestRegressor   
7               XGBRegressor   
4      DecisionTreeRegressor   
0                      Lasso   
2                 ElasticNet   

                                          Parameters CV score mean  
3  {'alpha': 0.01, 'coef0': 1, 'degree': 3, 'gamm...    -0.0701784  
1  {'alpha': 1.0, 'copy_X': True, 'fit_intercept'...    -0.0726033  
6  {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...    -0.0731346  
5  {'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...    -0.0760717  
7  {'objective': 'reg:squarederror', 'base_score'...    -0.0801293  
4  {'ccp_alpha': 0.0, 'criterion': 'mse', 'max_de...    -0.0970506  
0  {'alpha': 1.0, 'copy_X': True, 'fit_intercept'...     -0.399471  
2  {'alpha': 1.0, 'copy_X': True, 'fit_intercept'...     -0.399471

In [95]:
from sklearn import linear_model
ridge = KernelRidge()

In [96]:
ridge.fit(new_X_train, y_train)

KernelRidge()

In [97]:
gb_pred = gb_reg.predict(X_test_selected)
ridge_pred = ridge_reg.predict(X_test_selected)
xgb_pred = xgb_reg.predict(X_test_selected)
new_X_test = pd.DataFrame({'gb':gb_pred, 'ridge':ridge_pred, 'xgb':xgb_pred})
new_X_test

gb      ridge        xgb
0     11.683897  11.591410  11.655781
1     11.940919  11.997014  11.971737
2     12.123705  12.116505  12.068935
3     12.126110  12.200184  12.117094
4     12.169654  12.231946  12.217725
...         ...        ...        ...
1454  11.268089  11.353260  11.322364
1455  11.300778  11.285223  11.333141
1456  11.985878  11.932998  11.960294
1457  11.695552  11.707461  11.732255
1458  12.356756  12.269597  12.309108

[1459 rows x 3 columns]

In [98]:
ridge.predict(new_X_test)

array([11.6867664 , 11.93928723, 12.12402834, ..., 11.98755714,
       11.69524463, 12.3594811 ])

In [100]:
prediction = np.exp(ridge.predict(new_X_test))
prediction

array([118986.62819451, 153167.47779829, 184246.22915596, 184234.14827491,
       192495.489177  ])

In [ ]:
# 试一下调参

In [115]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

def neg_rmse(y_true, y_pred):
    return (-1) * np.sqrt(np.mean((y_true - y_pred) ** 2))
scorer = make_scorer(neg_rmse)

ridge_reg = KernelRidge(
    alpha=0.01,
    
)
parameters = {
    'kernel':['RBF', 'linear', 'polynominal', 'sigmoid', 'laplacian', ''],
    
}    
cv = GridSearchCV(estimator = ridge_reg, param_grid=parameters, scoring=scorer, n_jobs = -1, cv = 6)

cv.fit(new_X_train, y_train)

cv.best_params_

{'kernel': 'linear'}

# Make Predictions

In [82]:
prediction = (gb_reg.predict(X_test_selected) + xgb_reg.predict(X_test_selected) + ridge_reg.predict(X_test_selected))/3
prediction = np.exp(prediction)
prediction 

array([113970.60323509, 157927.2696859 , 180421.03991418, ...,
       156329.85902802, 121997.52627297, 222308.30288509])

In [119]:
prediction = np.exp(ridge_reg.predict(X_test_selected))

result = pd.DataFrame({'Id': test['Id'], 'SalePrice': prediction})
result.head()

Id      SalePrice
0  1461  108164.612140
1  1462  162269.562564
2  1463  182865.347698
3  1464  198825.738649
4  1465  205242.234839

In [120]:
result.to_csv('result.csv',index = False)